In [ ]:
!pip install transformers tensorflow

In [1]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

2025-08-16 13:34:06.660557: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-16 13:34:06.660679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-16 13:34:06.690654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-16 13:34:06.757620: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-16 13:34:08.222990: W tensorflow/compiler/tf2

In [2]:
# --- Load original model and tokenizer ---
model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)

2025-08-16 13:34:21.507688: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-16 13:34:21.514092: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
2025-08-16 13:34:22.305671: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 937

In [3]:
# --- Define the TF Module with the correct signature ---
class ToxicBertModule(tf.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    # This signature with [None, None] is key for dynamic batch and sequence length
    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, None], dtype=tf.int32, name='input_ids'),
        tf.TensorSpec(shape=[None, None], dtype=tf.int32, name='attention_mask')
    ])
    def __call__(self, input_ids, attention_mask):
        # The output must be a dictionary
        return {'logits': self.model(input_ids=input_ids, attention_mask=attention_mask).logits}


In [4]:
# --- Save the SavedModel ---
serving_module = ToxicBertModule(model)
tf.saved_model.save(serving_module, "saved_model_dynamic")

print("SavedModel with dynamic signature created.")

INFO:tensorflow:Assets written to: saved_model_dynamic/assets


INFO:tensorflow:Assets written to: saved_model_dynamic/assets


SavedModel with dynamic signature created.


In [5]:
# --- Convert to TFLite with settings to preserve dynamic shapes ---
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model_dynamic")

# IMPORTANT: Enable TF ops to handle operations not native to TFLite (common for BERT)
# This is crucial for keeping dynamic shapes.
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS    # Enable TensorFlow ops.
]

In [6]:
# Convert the model
tflite_model_dynamic = converter.convert()

# Save the new TFLite file
with open("toxic_bert_dynamic.tflite", "wb") as f:
    f.write(tflite_model_dynamic)

print("SUCCESS: Dynamic TFLite model saved as toxic_bert_dynamic.tflite")

2025-08-16 13:35:45.023301: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-08-16 13:35:45.023358: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-08-16 13:35:45.024274: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: saved_model_dynamic
2025-08-16 13:35:45.100892: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-08-16 13:35:45.100943: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: saved_model_dynamic
2025-08-16 13:35:45.234609: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2025-08-16 13:35:45.260071: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-08-16 13:35:46.113978: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: saved_model_dynami

SUCCESS: Dynamic TFLite model saved as toxic_bert_dynamic.tflite
